In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
import json, torch

c:\Users\won39\Desktop\DSBA_project1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 🔹 경로 및 모델명
file_path = "train_data/cleaned_dataset_half.json"
model_name = "beomi/KoAlpaca-Polyglot-5.8B"

# ✅ 1. 데이터 로드
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)

In [3]:
# ✅ 2. 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # LLaMA 계열에서 필수

# ✅ 3. 토크나이징 함수 정의
def tokenize(example):
    prompt = example["input"].strip()
    response = example["output"].strip()
    full_text = prompt + "\n" + response

    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize, remove_columns=["input", "output"])

Map: 100%|██████████| 6353/6353 [00:02<00:00, 2703.49 examples/s]


In [4]:
# ✅ 4. 양자화 설정 + 모델 로드
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=False  # CPU offload 금지
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)


The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
# ✅ 5. LoRA 설정 및 적용
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["attention.query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


model = get_peft_model(model, lora_config)


In [ ]:
# ✅ 6. 학습 설정
training_args = TrainingArguments(
    output_dir="train_data/koalpaca-lora-checkpoints",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    report_to="none",
    fp16=True
)

In [ ]:
# 7. 데이터 콜레이터 및 Trainer
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# 8. 학습 실행
trainer.train()

# 9. 모델 저장
trainer.save_model("train_data/koalpaca-lora-final")
tokenizer.save_pretrained("train_data/koalpaca-lora-final")

In [5]:
base_model = "beomi/KoAlpaca-Polyglot-5.8B"
lora_model_path = "train_data/koalpaca-lora-final"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(lora_model_path, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # 필수

# 베이스 모델 로드 + LoRA 적용
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    load_in_8bit=True
)
model = PeftModel.from_pretrained(model, lora_model_path)

model.eval()


OSError: train_data/koalpaca-lora-final is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`